In [6]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import pandas_market_calendars as mcal
from importlib import reload
import sys
import os
import time
from IPython.display import clear_output, display
from tqdm import tqdm

In [7]:
import functions; functions = reload(functions)
from functions import *

In [8]:
apikey = '3CenRhJBzNqh2_C_5S38pOyt3ozLvQDm'

In [4]:
# todo
# 1. test function for daily aggregates - none/errors/edge cases/before available
# 2. design pipeline for many jsons to csv
# 3. .db maybe?
# 4. if not, make csv
# 5. update/remake .py files to download and merge missing

In [15]:
baseurl = 'v3/reference/splits'
spliturl = 'https://api.polygon.io/v3/reference/splits?execution_date.lt=2024-06-01&apiKey=3CenRhJBzNqh2_C_5S38pOyt3ozLvQDm'

In [24]:
def option_handler(apikey='3CenRhJBzNqh2_C_5S38pOyt3ozLvQDm', noqm=False, **kwargs):
    kwargs['apiKey'] = apikey

    if 'from_date' in kwargs:
        kwargs['execution_date.gte'] = kwargs['from_date']
        del kwargs['from_date']
    if 'to_date' in kwargs:
        kwargs['execution_date.lte'] = kwargs['to_date']
        del kwargs['to_date']
        
    options = [f'{key}={value}' for key, value in kwargs.items()]
    output = '?' + '&'.join(options) if not noqm else '&' + '&'.join(options)
    
    return output
        
def make_request(baseurl, output='json', apikey='3CenRhJBzNqh2_C_5S38pOyt3ozLvQDm', **kwargs):
    if baseurl[:5] != 'https': baseurl = 'https://api.polygon.io/' + baseurl
    
    optionstring = option_handler(apikey, **kwargs) if 'cursor' not in baseurl else option_handler(apikey, noqm=True, **kwargs)
    url = baseurl + optionstring
    
    if output == 'json':
        return requests.get(url).json()
    else:
        return requests.get(url)

In [54]:
def get_splits(output='json', **kwargs):
    output = []
    response = make_request('v3/reference/splits', **kwargs)
    output.extend(response['results'])

    while ('next_url' in response.keys()):
        response = make_request(response['next_url'])
        output.extend(response['results'])
        
    return pd.DataFrame(output)

In [57]:
splits = get_splits(from_date='2024-05-01', to_date='2024-07-01')

In [59]:
splits.head()

,execution_date,split_from,split_to,ticker
0,2024-07-01,8.0,1.0,YGMZ
1,2024-07-01,17.0,1.0,RELI
2,2024-07-01,5.0,1.0,KORE
3,2024-07-01,10.0,1.0,JDST
4,2024-07-01,12.0,1.0,DARE


In [58]:
'WKHS' in splits.ticker

False

In [44]:
s = pd.DataFrame(make_request('v3/reference/splits', from_date='2024-01-01', to_date='2024-02-01', limit=100)['results'])

In [46]:
s.shape

(89, 4)

In [51]:
output = []
response = make_request('v3/reference/splits', limit=10, from_date='2024-01-01', to_date='2024-02-01')
output.extend(response['results'])

while ('next_url' in response.keys()):
    response = make_request(response['next_url'], limit=10)
    output.extend(response['results'])

In [ ]:
has_next = True
output = []
while has_next:
    response = make_request('v3/reference/splits', limit=1000)
    output.extend(response['results'])
    if 'next_url' in response.keys():
    has_next = response['has_next']
    cursor = response['cursor']

    

In [ ]:
option_handler(from_date='2024-03-28')

'?apiKey=3CenRhJBzNqh2_C_5S38pOyt3ozLvQDm&execution_date.gte=2024-03-28'

In [37]:
spls = make_request(baseurl, from_date='2024-01-01', limit=100)

In [36]:
spls.keys()

dict_keys(['results', 'status', 'request_id'])

In [38]:
spls.keys()

dict_keys(['results', 'status', 'request_id', 'next_url'])

In [30]:
pd.DataFrame(spls['results'])

,execution_date,split_from,split_to,ticker
0,2024-10-09,1.0,5.0,SONY
1,2024-10-03,1.0,10.0,LRCX
2,2024-09-30,1.0,3.0,KDCLF
3,2024-09-16,2.0,1.0,HSMCX
4,2024-09-16,2.0,1.0,HSLCX
...,...,...,...,...
95,2024-06-13,20.0,1.0,NAAS
96,2024-06-13,5.0,1.0,MWFEX
97,2024-06-13,10.0,1.0,DDNFF
98,2024-06-12,15.0,1.0,PEGY


In [31]:
spls['next_url']

'https://api.polygon.io/v3/reference/splits?cursor=YXA9MjAyNC0wNi0xMiZhcz1DRUdIRiZleGVjdXRpb25fZGF0ZS5ndGU9MjAyNC0wMS0wMSZleGVjdXRpb25fZGF0ZS5sdGU9MjAyNC0wNi0xMiZsaW1pdD0xMDAmb3JkZXI9ZGVzYyZzb3J0PWV4ZWN1dGlvbl9kYXRl'

In [149]:
a = get_splits(limit=2)

?limit=2


In [154]:
a['next_url']

'https://api.polygon.io/v3/reference/splits?cursor=YXA9MjAyNC0wOS0zMCZhcz1LRENMRiZleGVjdXRpb25fZGF0ZS5sdGU9MjAyNC0wOS0zMCZsaW1pdD0yJm9yZGVyPWRlc2Mmc29ydD1leGVjdXRpb25fZGF0ZQ'

In [179]:
a = make_request('v3/reference/splits', limit=2, ticker='GOOG')

In [183]:
requests.get(a['next_url'] + '&apiKey=3CenRhJBzNqh2_C_5S38pOyt3ozLvQDm').json()

{'results': [{'execution_date': '2014-03-27',
   'split_from': 1000,
   'split_to': 2002,
   'ticker': 'GOOG'}],
 'status': 'OK',
 'request_id': '64ced4ed718ae854c8de36a36ead6779'}

In [201]:
make_request(a['next_url'])


bla
&apiKey=3CenRhJBzNqh2_C_5S38pOyt3ozLvQDm


{'results': [{'execution_date': '2014-03-27',
   'split_from': 1000,
   'split_to': 2002,
   'ticker': 'GOOG'}],
 'status': 'OK',
 'request_id': '23604ecd99015c27beffc0933f2e2001'}

In [20]:
req = make_request('v3/reference/splits', output='')

In [22]:
req = req.json()

In [27]:
make_request(req['next_url'])

{'results': [{'execution_date': '2024-07-15',
   'split_from': 1,
   'split_to': 5,
   'ticker': 'USLM'},
  {'execution_date': '2024-07-15',
   'split_from': 1,
   'split_to': 10,
   'ticker': 'AVGO'},
  {'execution_date': '2024-07-10',
   'split_from': 1,
   'split_to': 2,
   'ticker': 'MITSY'},
  {'execution_date': '2024-07-09',
   'split_from': 1,
   'split_to': 2,
   'ticker': 'WSM'},
  {'execution_date': '2024-07-09',
   'split_from': 1,
   'split_to': 4,
   'ticker': 'ASCCY'},
  {'execution_date': '2024-07-03',
   'split_from': 8,
   'split_to': 1,
   'ticker': 'ASLN'},
  {'execution_date': '2024-07-01',
   'split_from': 10,
   'split_to': 1,
   'ticker': 'JDST'},
  {'execution_date': '2024-06-28',
   'split_from': 20,
   'split_to': 1,
   'ticker': 'YYAI'},
  {'execution_date': '2024-06-28',
   'split_from': 1,
   'split_to': 2,
   'ticker': 'TMRNF'},
  {'execution_date': '2024-06-28',
   'split_from': 1,
   'split_to': 5,
   'ticker': 'EBCOF'}],
 'status': 'OK',
 'request_id': 

In [23]:
req['next_url']

'https://api.polygon.io/v3/reference/splits?cursor=YXA9MjAyNC0wNy0xNSZhcz1VU0xNJmV4ZWN1dGlvbl9kYXRlLmx0ZT0yMDI0LTA3LTE1JmxpbWl0PTEwJm9yZGVyPWRlc2Mmc29ydD1leGVjdXRpb25fZGF0ZQ'

In [11]:
splits = pd.DataFrame(make_request('v3/reference/splits')['results'])

In [15]:
splits.shape

(10, 4)

In [14]:
splits.execution_date.min()

'2024-08-07'

In [5]:
date = '2020-01-01'
url = f'https://api.polygon.io/v2/aggs/grouped/locale/us/market/stocks/{date}?apiKey={apikey}'

In [6]:
response = requests.get(url)

In [7]:
response.json()

{'queryCount': 0,
 'resultsCount': 0,
 'adjusted': True,
 'status': 'OK',
 'request_id': 'a426fcea4aa5004a97f7d0e49adcb888'}

In [50]:
unauth = daily_agg('2018-01-02')
holiday = daily_agg('2020-01-01')
good = daily_agg('2020-06-01')

Attempted to request data past historical entitlements. Please upgrade your plan at https://polygon.io/pricing
No data available for 2020-01-01


In [17]:
mon = daily_agg('2024-06-03')
tue = daily_agg('2024-06-04')
wed = daily_agg('2024-06-05')

In [18]:
alljson = []
for json in [mon, tue, wed]:
    alljson.extend(json)

In [85]:
alldates = get_trading_days()
all = datelist_to_df(alldates)

In [147]:
import sqlite3

In [162]:
conn = sqlite3.connect('main.db')

In [163]:
all.head()

,T,v,vw,o,c,h,l,t,n
0,WB,981967.0,41.5953,41.47,41.61,41.93,41.02,1560801600000,11076.0
1,DLNGpB,14056.0,16.1327,16.51,16.00,16.51,15.90,1560801600000,102.0
2,WDC,4109647.0,36.3187,36.21,36.27,36.66,35.61,1560801600000,33259.0
3,GNTX,998643.0,23.4434,23.54,23.36,23.67,23.32,1560801600000,9896.0
4,BZQ,33935.5,104.5164,104.88,105.64,106.44,103.28,1560801600000,627.0


In [180]:
c = conn.cursor()

In [ ]:
c.execute('''
CREATE TABLE stonks (
    ticker TEXT,
    volume REAL,
    vw REAL,
    open REAL,
    close REAL,
    high REAL,
    low REAL,
    time INTEGER,
    n INTEGER
)
''')

In [189]:
info = c.execute('PRAGMA table_info(stonks)').fetchall()

In [190]:
info

[(0, 'ticker', 'TEXT', 0, None, 0),
 (1, 'volume', 'REAL', 0, None, 0),
 (2, 'vw', 'REAL', 0, None, 0),
 (3, 'open', 'REAL', 0, None, 0),
 (4, 'close', 'REAL', 0, None, 0),
 (5, 'high', 'REAL', 0, None, 0),
 (6, 'low', 'REAL', 0, None, 0),
 (7, 'time', 'INTEGER', 0, None, 0),
 (8, 'n', 'INTEGER', 0, None, 0)]

In [192]:
[i[1] for i in info]   

['ticker', 'volume', 'vw', 'open', 'close', 'high', 'low', 'time', 'n']

In [197]:
all.to_sql('stonks', conn, if_exists='append', index=False)

12678885